# Capacitated EOQ Model

In [16]:
# Load in the libraries that we'll use

import pandas as pd
import numpy as np

## Model Inputs

Feel free to modify these to understand how the model works. Then `Run All`. 

In [17]:
#This cell is for reading the system-wide inputs

total_hours = 8736  # this is in years
avail_regular_hours = 4264  # this is in years
avail_time_constraint = avail_regular_hours / total_hours  #expressed as a percentage
nominal_set_up_time = 10  #later we'll change this to a calculation
inv_holding_cost_percent = .2
cost_of_set_up_hour = 100


In [18]:
#This cell is for reading the product-specific inputs

product_input_table = pd.read_csv("product_input_v1.csv", index_col="Product ID")
product_input_table # display the table

,Demand (year)- m,Cost- c-j,Production Rate (hr),Set Up Scaler-- (q-j)
Product ID,,,,
1,5000,100,15,1
2,5000,200,15,1
3,8000,200,15,2
4,5000,400,15,2
5,5000,400,15,3
6,5000,500,15,3
7,5000,200,15,4
8,5000,500,15,4
9,5000,500,15,5


In [19]:
product_input_table["product_set_up_time_hrs"] = product_input_table["Set Up Scaler-- (q-j)"] * nominal_set_up_time
product_input_table["unit_set_up_cost"] = product_input_table["product_set_up_time_hrs"] * cost_of_set_up_hour
product_input_table

,Demand (year)- m,Cost- c-j,Production Rate (hr),Set Up Scaler-- (q-j),product_set_up_time_hrs,unit_set_up_cost
Product ID,,,,,,
1,5000,100,15,1,10,1000
2,5000,200,15,1,10,1000
3,8000,200,15,2,20,2000
4,5000,400,15,2,20,2000
5,5000,400,15,3,30,3000
6,5000,500,15,3,30,3000
7,5000,200,15,4,40,4000
8,5000,500,15,4,40,4000
9,5000,500,15,5,50,5000


## Internal calculations

In [20]:
# Rename the input table so that formulas are shorter and more readable

inputs_df = product_input_table.copy()
inputs_df.columns = ["demand", "cost", "rate", "scalar", "set_up_time", "set_up_cost"]

inputs_df

,demand,cost,rate,scalar,set_up_time,set_up_cost
Product ID,,,,,,
1,5000,100,15,1,10,1000
2,5000,200,15,1,10,1000
3,8000,200,15,2,20,2000
4,5000,400,15,2,20,2000
5,5000,400,15,3,30,3000
6,5000,500,15,3,30,3000
7,5000,200,15,4,40,4000
8,5000,500,15,4,40,4000
9,5000,500,15,5,50,5000


In [38]:
#create nasty calculation table
#create system-wide paramters  they will be mixed and matched depending on what needs to happen

cogs = (inputs_df["demand"] * inputs_df["cost"]).sum()

sys_S_nomimal_set_up_years = nominal_set_up_time / total_hours
sys_c_S_cost_of_set_up_years = cost_of_set_up_hour / total_hours * total_hours * total_hours #this was in my spreadsheet like this. need to figure out why


internal_calc_df = pd.DataFrame()  
internal_calc_df["Prod_rate_yr_r_j"] = inputs_df["rate"] * total_hours
internal_calc_df["m_over_r"] = inputs_df["demand"] / internal_calc_df["Prod_rate_yr_r_j"]
internal_calc_df["sqrt_2mqic_j"] = np.sqrt(2 * inputs_df["demand"] * inputs_df["scalar"] * inv_holding_cost_percent *inputs_df["cost"] )

c_sum = internal_calc_df["sqrt_2mqic_j"].sum()
alpha = internal_calc_df["m_over_r"].sum()
if alpha > avail_time_constraint:  #this means the problem is infeasible-- production time takes longer than available time
    infeasible_flag = True 
Else:
    infeasible_flag = False
    
lambda_var = (c_sum **2 * sys_S_nomimal_set_up_years) / (4*(avail_time_constraint - alpha)**2) - sys_c_S_cost_of_set_up_years 

x = sys_c_S_cost_of_set_up_years
y = sys_S_nomimal_set_up_years
internal_calc_df["Q-1 (S)"] = np.sqrt((2*inputs_df["demand"] * inputs_df["scalar"] * x * y)/(inv_holding_cost_percent * inputs_df["cost"]))
internal_calc_df["Q-2"] = ((c_sum * y) / (avail_time_constraint - alpha)) * np.sqrt((inputs_df["demand"] * inputs_df["scalar"]) / (2 * inv_holding_cost_percent * inputs_df["cost"])) 

print("sys_S_nomimal_set_up_years",sys_S_nomimal_set_up_years)
print("sys_c_S_cost_of_set_up_years",sys_c_S_cost_of_set_up_years)
print("alpha",alpha)
print("cogs",cogs)
print("lambda_var",lambda_var)
internal_calc_df.astype("float").round(3)


sys_S_nomimal_set_up_years 0.0011446886446886447
sys_c_S_cost_of_set_up_years 873600.0
alpha 0.40445665445665446
cogs 20600000
lambda_var 8854056.919427026


,Prod_rate_yr_r_j,m_over_r,sqrt_2mqic_j,Q-1 (S),Q-2
Product ID,,,,,
1,131040.0,0.038,447.214,707.107,2359.570
2,131040.0,0.038,632.456,500.000,1668.468
3,131040.0,0.061,1131.371,894.427,2984.646
4,131040.0,0.038,1264.911,500.000,1668.468
5,131040.0,0.038,1549.193,612.372,2043.447
6,131040.0,0.038,1732.051,547.723,1827.715
7,131040.0,0.038,1264.911,1000.000,3336.935
8,131040.0,0.038,2000.000,632.456,2110.463
9,131040.0,0.038,2236.068,707.107,2359.570


## Functions

In [22]:
# All the functions we plan on using

## Model

In [45]:
# Optimal Results of capaciated model

optimal_df = pd.DataFrame() 

if lambda_var <= 0:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-1 (S)"]
else:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-2"]

optimal_df["opt_setups_per_year"] = inputs_df["demand"] / optimal_df["q_star_order_size"]
optimal_df["opt_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / optimal_df["q_star_order_size"] 

opt_total_setup_percent = optimal_df["opt_time_in_setups_per_year"].sum()

print("opt_total_setup_percent",opt_total_setup_percent)
optimal_df.astype("float").round(4)

opt_total_setup_percent 0.08363858363858362


,q_star_order_size,opt_setups_per_year,opt_time_in_setups_per_year
Product ID,,,
1,2359.5697,2.1190,0.0024
2,1668.4677,2.9968,0.0034
3,2984.6458,2.6804,0.0061
4,1668.4677,2.9968,0.0069
5,2043.4473,2.4468,0.0084
6,1827.7148,2.7357,0.0094
7,3336.9355,1.4984,0.0069
8,2110.4633,2.3691,0.0108
9,2359.5697,2.1190,0.0121


In [50]:
# EOQ, Unconstrained Results -- theorhetical best, but not feasible

eoq_df = pd.DataFrame() 

eoq_df["q_star_eoq"] = np.sqrt((2 * inputs_df["demand"] * inputs_df["scalar"] * y * x) / (inv_holding_cost_percent * inputs_df["cost"] ))

eoq_df["eoq_setups_per_year"] = inputs_df["demand"] / eoq_df["q_star_eoq"]
eoq_df["eoq_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / eoq_df["q_star_eoq"] 

eoq_total_setup_percent = eoq_df["eoq_time_in_setups_per_year"].sum()

print("eoq_total_setup_percent",eoq_total_setup_percent)
eoq_df.astype("float").round(4)

eoq_total_setup_percent 0.2790965574587512


,q_star_eoq,eoq_setups_per_year,eoq_time_in_setups_per_year
Product ID,,,
1,707.1068,7.0711,0.0081
2,500.0000,10.0000,0.0114
3,894.4272,8.9443,0.0205
4,500.0000,10.0000,0.0229
5,612.3724,8.1650,0.0280
6,547.7226,9.1287,0.0313
7,1000.0000,5.0000,0.0229
8,632.4555,7.9057,0.0362
9,707.1068,7.0711,0.0405


In [54]:
# This is filling out the cost table, both EOQ and Optimal

cost_df = pd.DataFrame() 

#for optimal
cost_df["opt setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / optimal_df["q_star_order_size"]
cost_df["opt hold cost"] = optimal_df["q_star_order_size"] * inv_holding_cost_percent * inputs_df["cost"] / 2

#for EOQ
cost_df["eoq setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / eoq_df["q_star_eoq"]
cost_df["eoq hold cost"] = eoq_df["q_star_eoq"] * inv_holding_cost_percent * inputs_df["cost"] / 2

cost_df.astype("float").round(0)

,opt setup cost,opt hold cost,eoq setup cost,eoq hold cost
Product ID,,,,
1,2119.0,23596.0,7071.0,7071.0
2,2997.0,33369.0,10000.0,10000.0
3,5361.0,59693.0,17889.0,17889.0
4,5994.0,66739.0,20000.0,20000.0
5,7341.0,81738.0,24495.0,24495.0
6,8207.0,91386.0,27386.0,27386.0
7,5994.0,66739.0,20000.0,20000.0
8,9477.0,105523.0,31623.0,31623.0
9,10595.0,117978.0,35355.0,35355.0


## Results

In [82]:
# And here are the results!

print("----- Summary Stats ----------")
print("")
print("------Optimal Plan -----------")

total_holding_inventory_cost = round(cost_df['opt hold cost'].sum())
total_setup_cost = round(cost_df['opt setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (optimal_df["opt_time_in_setups_per_year"].sum() + alpha)
if hours_used >= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")

print("")
print("------EOQ Plan (Theoretical Best) -----------")

total_holding_inventory_cost = round(cost_df['eoq hold cost'].sum())
total_setup_cost = round(cost_df['eoq setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (eoq_df["eoq_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")




----- Summary Stats ----------

------Optimal Plan -----------
Total Holding Inventory Cost: $   813,607
Total Setup Cost:             $    73,067
Total Cost:                   $   886,674

Average Working Capital       $ 4,068,035
Inventory Turns                      5.1

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  4,264
Overtime Needed                         0

------EOQ Plan (Theoretical Best) -----------
Total Holding Inventory Cost: $   243,819
Total Setup Cost:             $   243,819
Total Cost:                   $   487,638

Average Working Capital       $ 1,219,095
Inventory Turns                     16.9

Total Hours- all time               8,736
Total Working Hours                 4,264
Working Hours Used                  5,972
Overtime Needed                     1,708
